In [ ]:
import cv2


img=cv2.imread("image.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow("grayimage",gray)
gaussian=cv2.GaussianBlur(img,(7,7),0)
cv2.imshow("gaussian blur",gaussian)
cannyedges=cv2.Canny(image= img, threshold1=50, threshold2=150)
cv2.imshow("cannyedges",cannyedges)
cv2.waitKey(0)
cv2.destroyAllWindows()




In [ ]:
import cv2
import numpy as np

img=cv2.imread("image.png")

width=img.shape[1]
height=img.shape[0]
imgsize=(width,height)
center=(width//2,height//2)

angle=60
scale=1
rotation_matrix=cv2.getRotationMatrix2D(center,angle,scale)
rotated_image=cv2.warpAffine(img,rotation_matrix,imgsize)
cv2.imshow("rotated_image",rotated_image)

scaling_factor=2
new_width=int(width/scaling_factor)
new_height=int(height/scaling_factor)
new_imgsize=(new_width,new_height)
scaled_image=cv2.resize(src= img, dsize=new_imgsize, interpolation=cv2.INTER_CUBIC)
cv2.imshow("scaled_image",scaled_image)

flipby=1
flipped_img=cv2.flip(img,flipby)
cv2.imshow("flipped_image",flipped_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
#splitting the dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train=X_train/255.0
X_test=X_test/255.0

X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
#designing the CNN
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

#compiling the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam',metrics=['accuracy'])

#training the model
model.fit(X_train, y_train, epochs=7, batch_size=64)

#evaluating the model
model.evaluate(X_test,y_test)


In [ ]:
import cv2
import numpy as np

# Load YOLOv3 model 
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Load class names
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Get output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

def detect_objects(img_path):
    img = cv2.imread(img_path)
    height, width, _ = img.shape

    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
            color = (0, 255, 0)
            
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("Detected Objects", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


img_paths = ["room.jpg", "forest.jpg", "kitchen.jpg"]

for path in img_paths:
    detect_objects(path)

'''
these models are used in object detection. they are used in autonomous detection for pedestrian identification,lane identification and obstacle
detection. the image passes through the yolo model and the output consists of prediction from the 3 detection layers. along with this, there are
two more features:- Confidence score and Class probability. confidence score indicates how sure it is that object is present inside the box and
class probability indicates that the detected object belongs to a particular class
'''  









In [ ]:
import numpy as np
import cv2

img = cv2.imread("image.png", 1)
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_blur=cv2.blur(gray_img,(3,3))
circles=cv2.HoughCircles(gray_blur,cv2.HOUGH_GRADIENT,1,20,param1=50,param2=30,minRadius=1,maxRadius=40)
circles=np.uint16(np.around(circles))
for pt in circles[0,:]:
    a,b,r=pt[0],pt[1],pt[2]
    cv2.circle(img,(a,b),r,(0,255,0),2)

triangles=[]
edged=cv2.Canny(gray_img,50,150)
contours,heirarchy=cv2.findContours(edged,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
for contour in contours:
    epsilon=0.04*cv2.arcLength(contour,True)
    approx=cv2.approxPolyDP(contour,epsilon,True)
    if len(approx)==3:
        triangles.append(approx)
        cv2.drawContours(img,[approx],0,(255,0,0),2)

for triangle in triangles:
    for circle in circles[0,:]:
        a,b,r=circle[0],circle[1],circle[2]
        inside=True
        for point in triangle:
            px,py=point[0]
            distance=np.sqrt((px-a)**2 + (py-b)**2)
            if distance>r:
                inside=False
                break
        if inside:
            cv2.drawContours(img,[triangle],0,(0,0,255),3)
            cv2.circle(img,(a,b),r,(0,255,255),3)

cv2.imshow("final image",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

